In [1]:
from sklearn import linear_model
from sklearn import datasets ## Importa datasets de scikit-learn
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score, roc_auc_score
import numpy as np
import pandas as pd

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#Cargar base de datos
# define data usando Dataframe
datos = pd.read_csv('p1_dataset_elpino_raw.csv', on_bad_lines='skip')
df = pd.DataFrame(datos)
#datos.head()
#print(datos)
#print(df.dtypes)
#datos.fillna(value = 0)

#Exited = 1 que el cliente se va del banco

In [ ]:

 
##Ajuste de variables metodo label encoder
from sklearn.preprocessing import LabelEncoder

#Tranformar la variable Geography a categoría
#Definimos France = 0, Germany = 1 y Spain = 2
sex = LabelEncoder()
X[:,1] = sex.fit_transform(X[:,1])